# Algorithm

In [6]:
class HierarchicalClustering:
    
    def __init__(self, alpha = 1.01):
        ''' '''
        self.alpha = alpha
        
    def fit(self, x):
        ''' '''
        
    def fit_transform(self, x): 
        ''' '''
        return x
        
    def print_name(self):
        print('Hierarchical clustering')
        
hc = HierarchicalClustering()
hc.print_name()

Hierarchical clustering
